# <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 2px; color:#145f19; font-size:100%; text-align:center;padding: 0px; border-bottom: 3px solid #145f19">Main purpose:</p>
<a href="https://machinelearningknowledge.ai/keras-lstm-layer-explained-for-beginners-with-example/"><img src="https://github.com/vainero/Files/blob/main/LSTM_logo.png?raw=true" width="15%" height="10%" align="right" /></a>


* In this notebook, we will focus on the **[Long Short-Term Memory (LSTM)](https://bom.so/mGR5Nk)** layer model to predict if the stock of Netflix will go up or down in the period between 30/09/2021 - 29/09/2023 and give the forecast about the stock in next 30 days.

* Historical daliy **[Netflix (NFLX)](https://finance.yahoo.com/)** stock price was collected from Yahoo Finance.

# <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 2px; color:#145f19; font-size:100%; text-align:center;padding: 0px; border-bottom: 3px solid #145f19">Import Libraries</p>

In [ ]:
# Import all libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
%matplotlib inline

# <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 2px; color:#145f19; font-size:100%; text-align:center;padding: 0px; border-bottom: 3px solid #145f19">Loading Data</p>

In [ ]:
# Load data from file .csv
data = pd.read_csv('/content/NFLX.csv')
data.head()

<div style="border-radius:5px;
            background-color:#ffffff;
            border-style: solid;
            border-color: #145f19;
            letter-spacing:1px;             
            font-size:16px;
            padding: 10px;">

|<font color='#145f19'>Price</font> | <font color='#145f19'>Description</font>
|:--- | :---|
|<font color='darkred'>Open</font>| The price of the stock at the beginning of the trading day
|<font color='darkred'>High</font>| The highest price of the stock on the trading day
|<font color='darkred'>Low</font> | The lowest price of the stock on the trading day
|<font color='darkred'>Close</font>| The last price of the stock before the trading day ends

In [ ]:
# Convert the type of "Date" column into date-time
data["Date"] = pd.to_datetime(data["Date"])
print (data.dtypes)

In [ ]:
# Add the "Year", 'Month' and "Day" columns
data["Year"] = data["Date"].apply(lambda x : str(x)[:4])
data["Month"] = data["Date"].apply(lambda x : str(x)[5:7])
data["Day"] = data["Date"].apply(lambda x : str(x)[8:10])

data.head()

# <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 2px; color:#145f19; font-size:100%; text-align:center;padding: 0px; border-bottom: 3px solid #145f19">Feature Extraction</p>

In [ ]:
# Print the number of trading days
print(f'Total number of trading days: {data.shape[0]}')
print(f'Total number of fields: {data.shape[1]}')
print('\n')

# Check the Null and NA values
print(f'Null values: {data.isnull().values.sum()}')
print(f'NA values: {data.isna().values.any()}')

* We will select from the dataset the **Open** column represents the starting price of the stock for a particular day.

In [ ]:
# Create a new dataframe with only the Open column and convert it into a NumPy array
data = data['Open'].values

# Reshape the data
data = data.reshape(-1, 1)

In [ ]:
# Split the data into training and testing sets
dataset_train = np.array(data[:int(data.shape[0]*0.8)])
dataset_test = np.array(data[int(data.shape[0]*0.8):])

* To produce the best-optimized results, we are required to scale the data.

* For this task, we are leveraging scikit-learn library’s **MinMaxScaler()** for converting the input values between 0 to 1.

In [ ]:
# Scale the data
scaler = MinMaxScaler(feature_range=(0,1))

dataset_train = scaler.fit_transform(dataset_train)
dataset_test = scaler.transform(dataset_test)

* We will create the helper function with 50 timesteps. In other words, for the features(x) we'll append the last 50 prices, and for the label(y) we'll append the next price.

* After that we will convert it into a NumPy array.

In [ ]:
# Create the function that will help us to create the datasets
def create_dataset(df):
    x = []
    y = []
    for i in range(50, df.shape[0]):
        x.append(df[i-50:i, 0])
        y.append(df[i, 0])
    x = np.array(x)
    y = np.array(y)
    return x,y

In [ ]:
# Create the training and testing data by calling our function for each one
x_train, y_train = create_dataset(dataset_train)
x_test, y_test = create_dataset(dataset_test)

In [ ]:
# Reshape the data to make it a 3D array in order to use it in LSTM Layer
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 2px; color:#145f19; font-size:100%; text-align:center;padding: 0px; border-bottom: 3px solid #145f19">Building Model</p>
* For the LSTM layer, we add **96 units** that represent the dimensionality of the output space.

* The **return_sequences parameter** is set to **True** for returning the last output in output.

* For the Dropout layers, we specify the percentage of layers is **20%** that should be dropped.

* The next step is to add the Dense layer.

In [ ]:
#Build the model
model = Sequential()
model.add(LSTM(units=96, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=96, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=96, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=96))
model.add(Dropout(0.2))
model.add(Dense(units=1))

In [ ]:
# Summary of the model
model.summary()

In [ ]:
# Reshape the feature for the LSTM layer
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) # for expecting 3 dimentions, not 2
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

* We will compile the model with the help of **adam** optimizer.

* The error is computed using **mean_squared_error**.

In [ ]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 2px; color:#145f19; font-size:100%; text-align:center;padding: 0px; border-bottom: 3px solid #145f19">Training Model</p>
* With our model and data constructed, we can now train the model.

In [ ]:
# Define modifiable training hyperparameters
epochs = 100
batch_size = 32

# start the training
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

# Print out the final loss
final_loss = history.history['loss'][-1]
print(f"Final Loss: {final_loss}")

In [ ]:
#Extract loss values from the training history
loss = history.history['loss']

#Create a range of x values for the epochs
epochs_range = range(1, epochs + 1)

#Plot the loss and accuracy in the line graph
plt.figure(figsize=(10, 6))
plt.plot(epochs_range, loss, label='Training Loss', color='blue')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
#Save the model
model.save('stock_prediction')

In [ ]:
# Load the model
model = load_model('stock_prediction')

# <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 2px; color:#145f19; font-size:100%; text-align:center;padding: 0px; border-bottom: 3px solid #145f19">Prediction Model</p>

In [ ]:
#Prediction model
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
y_test_scaled = scaler.inverse_transform(y_test.reshape(-1, 1))

#Plotting Results
fig, ax = plt.subplots(figsize=(20,15))
ax.plot(y_test_scaled, color='blue', linewidth=2, label='Original price')
plt.plot(predictions, color='orange', linewidth=2.5, label='Predicted price')
plt.title('Netflix Stock Price Prediction')
plt.legend();

# <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 2px; color:#145f19; font-size:100%; text-align:center;padding: 0px; border-bottom: 3px solid #145f19">Prediction for Next 30 days</p>

In [ ]:
#Next 30 days prediction
time_step=100
input_data=np.array(data[-time_step:])
input_data=input_data.reshape([input_data.shape[0],1])

# define predictions
def predict(data,days=30):
    data=scaler.transform(data)
    predictions=[]
    i=1
    while(i<=days):
        nxtday=model.predict([data],verbose=0)
        predictions.append(scaler.inverse_transform(nxtday)[0])
        data[:-1]=data[1:]
        data[-1]=nxtday[0]
        i+=1
    return np.array(predictions).squeeze()

#prediction
days=30
predictions=predict(input_data,days)

trainPredPlot=np.zeros(shape=[len(input_data)+1+days])
trainPredPlot[:]=np.nan
trainPredPlot[len(input_data)]=input_data[-1]
trainPredPlot[len(input_data)+1:]=predictions
df_=input_data

plt.plot(df_,label="Actual close price")
plt.plot(trainPredPlot,label="Predicted close price")
plt.legend()
plt.show()

# <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 2px; color:#145f19; font-size:100%; text-align:center;padding: 0px; border-bottom: 3px solid #145f19">Print out all metrics for evaluate the model</p>

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

# Calculate the RMSE
rmse = math.sqrt(mean_squared_error(y_test_scaled[:30], predictions))/ 100

# Calculate the MAE
mae = mean_absolute_error(y_test_scaled[:30], predictions)/ 100

# Print the results
print(f'MAE: {mae:.4f}')
print(f'RMSE: {rmse:.4f}')
